# Linea de Muerte Z

##  1. Big Picture

* Preprocesamiento
   * Se quitan del datataset los *envenenados* atributos  mprestamos_personales y cprestamos_personales
   * Se agregan lags y delta_lags de orden 1 y 2
* Modelado no se optimizan hiperarámetros
* Produccion
   * Entrenamieento final
      * Se entrena en {202101, 202102, 202103, 202104}
      * Se hace un **muy agresivo** undersampling de **0.10** de los "CONTINUA"
      * POS = {"BAJA+1", "BAJA+2"}
      * librería  *zLightGBM*
         * **50** canaritos se agregan al comienzo del dataset
         * gradient_bound se deja en su default de  **0.1**
   * Clasificacion
      * Se corta en 11000  envios

---
Resultados :
* ganancia de 383.371 M en el Private Leaderboard ( 11.000 en el Public )
* utiliza 12 GB de memoria RAM
* corre en  6 minutos

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-05 09:35:37 -03"

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,660709,35.3,1462257,78.1,NA,1073932,57.4
Vcells,1228649,9.4,8388608,64.0,32768,2010932,15.4


In [ ]:
PARAM <- list()
PARAM$experimento <- "zmuerte-25-UVA"
PARAM$semilla_primigenia <- 450343

In [ ]:
#setwd('/Users/manumoreira/Repos/dmeyf2025/competencia2/')
#BASE_PATH <- getwd()
#DATA_PATH <- "../../data/competencia_02_crudo.csv.gz"

#exp_name <- sprintf("exp%s_seed_%d", PARAM$experimento, PARAM$semilla_primigenia)
#exp_dir <- file.path(BASE_PATH, "results", exp_name)
#dir.create(exp_dir, recursive = TRUE, showWarnings = FALSE)
#setwd(exp_dir)

cat("Working directory:", getwd(), "\n")
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))
DATA_PATH = "https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_01_crudo.csv"

Working directory: /Users/manumoreira/Repos/dmeyf2025/competencia2/results/expzmuerte-10-manu_seed_450343 


## Preprocesamiento

### Generacion de la clase_ternaria

In [ ]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread(DATA_PATH)

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-05 09:56:35 -03"

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,770964,41.2,1462257,78.1,NA,1462257,78.1
Vcells,722154630,5509.7,1017411198,7762.3,32768,846003906,6454.5


[1] "2025-11-05 09:56:45 -03"

### Eliminacion de Features

La auténtica  **salsa mágica**  de este script es la eliminación de estos dos atributos del dataset ya que tran problemas con el Data Drifting
* mprestamos_personales
* cprestamos_personales

el problema con estos campos se detectó manualmente mediante un analisis exploratorio de datos y análisis de corridas de LightGBM en meses previos.

In [23]:
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

Sys.time()

Warning message in `[.data.table`(dataset, , `:=`(mprestamos_personales, NULL)):
"Tried to assign NULL to column 'mprestamos_personales', but this column does not exist to remove"
Warning message in `[.data.table`(dataset, , `:=`(cprestamos_personales, NULL)):
"Tried to assign NULL to column 'cprestamos_personales', but this column does not exist to remove"


[1] "2025-11-05 18:20:32 -03"

### Data features

In [ ]:
# Features nuevos
dataset[, x_margen_total := mactivos_margen + mpasivos_margen] # Entró
dataset[, x_pct_margen_activos := mactivos_margen / pmax(margen_total, 1)] # Entró
dataset[, x_trx_visa_por_cuenta := ctarjeta_visa_transacciones / pmax(ctarjeta_visa, 1)] # Entró
dataset[, x_ticket_visa := mtarjeta_visa_consumo / pmax(ctarjeta_visa_transacciones, 1)] # Entró
dataset[, x_pct_dolares := (mcaja_ahorro_dolares + mplazo_fijo_dolares + minversion1_dolares) / 
         pmax(mcuentas_saldo + mplazo_fijo_pesos + mplazo_fijo_dolares, 1)] # Entró
dataset[, x_debitos_auto_total := ccuenta_debitos_automaticos + 
                                  ctarjeta_visa_debitos_automaticos + 
                                  ctarjeta_master_debitos_automaticos] # Entró
dataset[, x_monto_debito_promedio := (mcuenta_debitos_automaticos + 
                                     mtarjeta_visa_debitos_automaticos + 
                                     mttarjeta_master_debitos_automaticos) / 
                                    pmax(debitos_auto_total, 1)] # Entró
dataset[, x_patrimonio_neto := 
  (mcuentas_saldo + 
   mcaja_ahorro + mcaja_ahorro_adicional + mcaja_ahorro_dolares +
   mcuenta_corriente + mcuenta_corriente_adicional +
   mplazo_fijo_pesos + mplazo_fijo_dolares +
   minversion1_pesos + minversion1_dolares + minversion2)
  -
  (mprestamos_prendarios + mprestamos_hipotecarios +
   Master_msaldototal + Visa_msaldototal)]
dataset[, x_mes := foto_mes %% 100]   
dataset[, x_quarter := ceiling(x_mes / 3)]
dataset[, x_lifecycle_stage := cliente_antiguedad / pmax(cliente_edad * 12, 1)]
dataset[, x_rentabilidad_por_producto := mrentabilidad / pmax(cproductos, 1)]
dataset[, x_product_complexity := (tcuentas + ctarjeta_visa + ctarjeta_master + cprestamos_personales + 
                                  cprestamos_hipotecarios + cseguro_vida + cseguro_auto) / 7]
dataset[, x_liquidity_stress := (Master_delinquency + Visa_delinquency + 
                                mcomisiones_mantenimiento + mcomisiones_otras) / 
         pmax(mcuentas_saldo, 1)]
dataset[, x_wealthy_active := x_patrimonio_neto * log1p(x_engagement_score)]
dataset[, x_wealth_premium_behavior := x_patrimonio_neto * x_ticket_visa]
dataset[, x_dollarized_engaged := x_pct_dolares * x_engagement_score]
dataset[, x_wealth_automated := x_patrimonio_neto * x_debitos_auto_total]
dataset[, x_dollar_visa_premium := x_pct_dolares * x_ticket_visa]
dataset[, x_dollar_visa_activity := x_pct_dolares * x_trx_visa_por_cuenta]
dataset[, x_visa_auto_premium := x_ticket_visa * x_monto_debito_promedio]
dataset[, x_auto_premium := x_debitos_auto_total * x_ticket_visa * x_pct_margen_activos]

In [ ]:
# Cargo datos UVA
uva_dt <- fread("~/buckets/b1/datasets/UVA.csv")
uva_dt[, Fecha := as.Date(Fecha, format = "%d/%m/%Y")]
uva_dt[, year_month := format(Fecha, "%Y%m")]

# Tomo el primer valor de cada mes
uva_first_day <- uva_dt[, .SD[1], by = year_month][, .(year_month, uva_value = Valor)]

# Cast
uva_first_day[, uva_value := as.numeric(gsub(",", ".", uva_value))]
uva_first_day[, year_month := as.character(year_month)]

convert_to_uva <- function(dt, monetary_vars, uva_ref) {
  # Create a temporary character version of foto_mes
  temp_dt <- copy(dt)
  temp_dt[, foto_mes_char := as.character(foto_mes)]
  
  # Perform the join
  dt_result <- uva_ref[temp_dt, on = .(year_month = foto_mes_char)]
  
  # Create new _uva features for each monetary variable
  for (var in monetary_vars) {
    uva_var <- paste0(var, "_uva")
    dt_result[, (uva_var) := get(var) / uva_value]
  }
  
  # Remove the temporary columns
  dt_result[, c("year_month", "uva_value", "foto_mes_char") := NULL]
  
  return(dt_result)
}

In [ ]:
monetary_vars <- c("mrentabilidad", "mrentabilidad_annual", "mcomisiones", "mactivos_margen", "mpasivos_margen", 
                "mcuenta_corriente_adicional", "mcuenta_corriente", "mcaja_ahorro", "mcaja_ahorro_adicional", 
                "mcaja_ahorro_dolares", "mcuentas_saldo", "mautoservicio", "mtarjeta_visa_consumo", "mtarjeta_master_consumo", 
                "mprestamos_prendarios", "mprestamos_hipotecarios", "mplazo_fijo_dolares", 
                "mplazo_fijo_pesos", "minversion1_pesos", "minversion1_dolares", "minversion2", "mpayroll", "mpayroll2", 
                "mcuenta_debitos_automaticos", "mttarjeta_visa_debitos_automaticos", "mttarjeta_master_debitos_automaticos", 
                "mpagodeservicios", "mpagomiscuentas", "mcajeros_propios_descuentos", "mtarjeta_visa_descuentos", 
                "mtarjeta_master_descuentos", "mcomisiones_mantenimiento", "mcomisiones_otras", "mforex_buy", "mforex_sell", 
                "mtransferencias_recibidas", "mtransferencias_emitidas", "mextraccion_autoservicio", "mcheques_depositados", 
                "mcheques_emitidos", "mcheques_depositados_rechazados", "mcheques_emitidos_rechazados", "matm", "matm_other", 
                "Master_mfinanciacion_limite", "Master_msaldototal", "Master_msaldopesos", "Master_msaldodolares", 
                "Master_mconsumospesos", "Master_mconsumosdolares", "Master_mlimitecompra", "Master_madelantopesos", 
                "Master_madelantodolares", "Master_mpagado", "Master_mpagospesos", "Master_mpagosdolares", 
                "Master_mconsumototal", "Master_mpagominimo", "Visa_mfinanciacion_limite", "Visa_msaldototal", 
                "Visa_msaldopesos", "Visa_msaldodolares", "Visa_mconsumospesos", "Visa_mconsumosdolares", 
                "Visa_mlimitecompra", "Visa_madelantopesos", "Visa_madelantodolares", "Visa_mpagado", "Visa_mpagospesos", 
                "Visa_mpagosdolares", "Visa_mconsumototal", "Visa_mpagominimo")
dataset <- convert_to_uva(dataset, monetary_vars, uva_first_day)
#Borro variables originales
dataset[, (monetary_vars) := NULL]

In [ ]:
colnames(dataset)

### Feature Engineering Historico

A partir del dataset crudo, se generan los lags y delta de orden 1 y 2

In [10]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

[1] "2025-11-05 09:57:49 -03"

## Modelado

### Optimizacion de Hipeparámetros

Este script no hace optimización de hiperparámetros
<br> **No** se llama a una Bayesian Optimization, ese paso se saltea.
<br> No hace falta hacer particiones <train, validate, test>,  tampoco se hace un k-fold cross validation

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los cuatro meses  {202101, 202102, 202103, 202104}, esta es una meticulosa decisión.
* Se hace un muy agresivo undersampling del **0.10** = 10% de la clase mayoritaria (los "CONTINUA" )
* Obviamente los datos donde se aplica el modelo es el mes  {202106}
* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}, , esta es una *enorme* decisión.
* No se optmizan los hiperparámetros de LightGBM, sino que se llama a la libreria *zLightGBM*
* Para *zLightGBM*  se crean **50** canaritos, esta es una decisión enorme !

In [ ]:
# training y future
Sys.time()

PARAM$train_final$meses <- c(202101, 202102, 202103, 202104)
PARAM$train_final$undersampling <- 0.25

PARAM$future <- c(202106)

[1] "2025-11-05 10:02:03 -03"

### Final Training Strategy

Se entrena en los cuatro meses {202101, 202102, 202103, 202104}
<br>Se hace un muy agresivo undersampling del 10% de la clase mayoritaria (los "CONTINUA" )

In [12]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$meses]

In [13]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [14]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Model

In [21]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

Loading required package: zlightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'zlightgbm'"
Warning message in install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", :
"installation of package '/var/folders/v5/kbcdcc412tb1kkysqm2lplxm0000gq/T//RtmpaNLqy8/downloaded_packages/zlightgbm_4.6.0.99.tar.gz' had non-zero exit status"
Loading required package: zlightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'zlightgbm'"


[1] "2025-11-05 10:05:30 -03"

In [ ]:
library(zlightgbm)

Un hiperparámetro de  *zLightGBM* es la cantidad de canaritos, en este caso se decidió (con tests realizados en {202101, 202102} fijarlo en 50
<br> esta es una importante decisión que debe tomarse
<br> en la version actual de *zLightGBM* los canaritos deben agregarse por fuera al dataset, y mandatoriamente deben ser las primeras columnas del mismo
<br> durante noviembre, si hay suerte, se liberará una versiónn que lo haga automáticamente internamente en el código C++ de la librería

In [ ]:
# canaritos
PARAM$qcanaritos <- 60

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

for( i in seq(PARAM$qcanaritos) ){
  dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
}

# las columnas canaritos mandatoriamente van al comienzo del dataset
cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )

Sys.time()

[1] "2025-10-31 14:59:04 UTC"

In [13]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)

In [14]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 71714 columnas 802 


[1] "2025-10-31 14:59:05 UTC"

Nuevos hiperparámetros en  *zLightGBM*
* canaritos , entero >=0,  cantidad de atributos del dataset que se consideran como canaritos, mandatoriamente en la version actual deben estar al comienzo.
* gradient_bound , numero real, >=0  cota que se pone a los scores de las hojas de los arboles, es un learning_rate adaptativo

En el caso que  canaritos==0  y  gradient_bound==0  entonces  xLightGBM se comporta exactamente igual a  LightGBM

Tener en cuenta que el hiperparámetro  min_data_in_leaf se está dejando en el valor de 20 que es el default de LightGBM
<br> realmente valdria la pena experimentar con distintos valores de  min_data_in_leaf, hay potencial de mejora !

In [15]:
# definicion de parametros, los viejos y los nuevos

PARAM$lgbm <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= 20L,  #este ya es el valor default de LightGBM

  num_iterations= 9999L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= 1.0,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= 0.1  # default de zLightGBM
)

Sys.time()

[1] "2025-10-31 14:59:05 UTC"

####  Entrenamiento del modelo

Aqui se ejecuta  *zLightGBM*
<br> no se utilizan hiperparámetros optimos para controlar el overfitting
<br> zLightGBM  sabe cuando no hacer un split
<br> si al hacer el n-simo arbol, no puede hacer el split de la raiz, detiene el crecimiento del ensemble y termina
<br>
<br> claramente el  min_data_in_leaf=20 que es el default de LightGBM esta jugando un papel importante

In [16]:
# entreno el modelo

modelo_final <- lgb.train(
  data= dtrain_final,
  param= PARAM$lgbm
)

Sys.time()

[1] "2025-10-31 15:03:22 UTC"

In [17]:
# grabo el modelo generado, esto pude ser levantado por LighGBM en cualquier maquina
lgb.save(modelo_final, file="zmodelo.txt")

# grabo un dataset que tiene el detalle de los arboles de LightGBM
tb_arboles <- lgb.model.dt.tree(modelo_final)
fwrite(tb_arboles, file="tb_arboles.txt", sep="\t")

cat("cantidad arbolitos=", tb_arboles[, max(tree_index)+1],"\n" )
cat("summary de las hojas de los arboles")
summary( tb_arboles[, list(hojas=max(leaf_index, na.rm=TRUE)+1), tree_index][,hojas])

Sys.time()

cantidad arbolitos= 285 
summary de las hojas de los arboles

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    3.0   167.0   358.0   373.4   534.0   982.0 

[1] "2025-10-31 15:03:32 UTC"

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [18]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future]

# penosamente, en la versión actual de zLightGBM  los campos canaritos
#  aunque no se utilizan para nada, también deben estar en el dataset donde se hace el predict()
filas <- nrow(dfuture)

for( i in seq(PARAM$qcanaritos) ){
  dfuture[, paste0("canarito_",i) := runif( filas) ]
}

prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE]),
)

In [19]:
# tabla de prediccion, puede ser util para futuros ensembles
#  ya que le modelo ganador va a ser un ensemble de LightGBMs

tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

In [ ]:
# Feature importance
tb_importancia <- as.data.table(lgb.importance(modelo_final))
fwrite(tb_importancia, file = "impo.txt", sep = "\t")

cat("\nTop 10 most important features:\n")
print(head(tb_importancia, 10))

# Save model
lgb.save(modelo_final, "modelo.txt")
cat("\nModel and importance saved\n")

### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+2"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [20]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

envios <- 11000
tb_prediccion[, Predicted := 0L] # seteo inicial a 0
tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)

### Subida a Kaggle

In [21]:
# subida automática a Kaggle, solo tiene sentido para la Primera Competencia

comando <- "kaggle competitions submit"
UBA_comp <- "-c dm-ey-f-2025-primera"
arch <- paste("-f", archivo_kaggle)

mensaje <- paste0("-m 'under=", PARAM$train_final$undersampling,
  "  cana=", PARAM$qcanaritos,
  "  gb=", PARAM$lgbm$gradient_bound,
  "  ff=", PARAM$lgbm$feature_fraction,
  "  mdil=", PARAM$lgbm$min_data_in_leaf,
  "  lr=", PARAM$lgbm$learning_rate, "'"
)

linea <- paste(comando, UBA_comp, arch, mensaje)
salida <- system(linea, intern=TRUE)
cat(salida)

Successfully submitted to DMEyF 2025 Primera

In [22]:
Sys.time()

[1] "2025-10-31 15:03:38 UTC"